# Definición de Arquitectura GAN (Generador y Discriminador)

Este cuaderno define las clases para el Generador y el Discriminador basados en LSTM.
El objetivo es validar las dimensiones de entrada y salida antes de pasar al bucle de entrenamiento.

In [ ]:
import torch
import torch.nn as nn

## 1. Definición del Generador

El Generador toma un vector de ruido aleatorio (espacio latente) y genera una secuencia de precios sintéticos.

In [ ]:
class Generator(nn.Module):
    """
    Generador basado en LSTM para crear secuencias de series temporales sintéticas.
    """
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int = 1, num_layers: int = 1):
        """
        Inicializa el Generador.

        Args:
            input_dim (int): Dimensión del espacio latente (ruido).
            hidden_dim (int): Dimensión oculta de la LSTM.
            output_dim (int): Dimensión de salida (1 para precio univariante).
            num_layers (int): Número de capas LSTM.
        """
        super(Generator, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Paso hacia adelante del Generador.

        Args:
            x (torch.Tensor): Tensor de entrada de ruido con forma (batch_size, seq_len, input_dim).

        Returns:
            torch.Tensor: Secuencia generada con forma (batch_size, seq_len, output_dim).
        """
        # LSTM output shape: (batch_size, seq_len, hidden_dim)
        lstm_out, _ = self.lstm(x)
        
        # Mapear a la dimensión de salida
        output = self.linear(lstm_out)
        
        # Aplicar Sigmoid para escalar entre 0 y 1
        return self.sigmoid(output)

## 2. Definición del Discriminador

El Discriminador toma una secuencia (real o falsa) y clasifica si es auténtica.

In [ ]:
class Discriminator(nn.Module):
    """
    Discriminador basado en LSTM para clasificar secuencias como Reales o Falsas.
    """
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int = 1, num_layers: int = 1):
        """
        Inicializa el Discriminador.

        Args:
            input_dim (int): Dimensión de entrada (1 para precio univariante).
            hidden_dim (int): Dimensión oculta de la LSTM.
            output_dim (int): Dimensión de salida (1 para probabilidad).
            num_layers (int): Número de capas LSTM.
        """
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Paso hacia adelante del Discriminador.

        Args:
            x (torch.Tensor): Secuencia de entrada con forma (batch_size, seq_len, input_dim).

        Returns:
            torch.Tensor: Probabilidad de ser real con forma (batch_size, 1).
        """
        # LSTM output: (batch_size, seq_len, hidden_dim)
        # Solo necesitamos el último estado oculto para clasificación de secuencia completa
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Usamos el último estado oculto de la última capa: h_n[-1]
        # h_n shape: (num_layers, batch_size, hidden_dim) -> tomamos el último layer
        last_hidden = h_n[-1]
        
        output = self.linear(last_hidden)
        return self.sigmoid(output)

## 3. Verificación de Dimensiones (Sanity Check)

Probamos los modelos con datos aleatorios para asegurar que las dimensiones fluyen correctamente.

In [ ]:
# Parámetros de prueba
BATCH_SIZE = 64
SEQ_LEN = 24
NOISE_DIM = 10
HIDDEN_DIM = 32
FEATURE_DIM = 1

# Instanciar modelos
generator = Generator(input_dim=NOISE_DIM, hidden_dim=HIDDEN_DIM, output_dim=FEATURE_DIM)
discriminator = Discriminator(input_dim=FEATURE_DIM, hidden_dim=HIDDEN_DIM, output_dim=1)

print("Modelos instanciados correctamente.")

# Crear ruido aleatorio
# Input del Generador: (batch, seq_len, noise_dim)
noise = torch.randn(BATCH_SIZE, SEQ_LEN, NOISE_DIM)
print(f"Ruido de entrada: {noise.shape}")

# Generar datos falsos
fake_data = generator(noise)
print(f"Datos generados (Fake): {fake_data.shape}")

# Pasar por el Discriminador
# El discriminador espera (batch, seq_len, feature_dim)
# fake_data ya tiene esa forma (64, 24, 1)
disc_out = discriminator(fake_data)
print(f"Salida del Discriminador: {disc_out.shape}")

# Verificaciones
assert fake_data.shape == (BATCH_SIZE, SEQ_LEN, FEATURE_DIM), "Error en dimensiones del Generador"
assert disc_out.shape == (BATCH_SIZE, 1), "Error en dimensiones del Discriminador"
print("\n¡Prueba de dimensiones exitosa!")